In [1]:
import base64, IPython

In [2]:
from collections import UserList

In [3]:
from vdom import *
from vdom.svg import *

In [4]:
element = div('%s', style={'flex': '1'})._repr_html_()
row = div('%s', style={'display': 'flex', 'flex-direction': 'row'})._repr_html_()
column = div('%s', style={'display': 'flex', 'flex-direction': 'column'})._repr_html_()

In [5]:
class Container(UserList):
    def _repr_html_(self, body=""):
        for object in self:
            output, metadata = IPython.get_ipython().display_formatter.format(object)
            for key in list(output):
                if key.startswith('image'):
                    encoded = output.pop(key)
                    if key.endswith('xml'):
                        output['text/html'] = encoded
                    else:
                        if isinstance(encoded, bytes):
                            encoded = base64.b64encode(encoded).decode('utf-8')
                        output['text/html'] = img(src=f"data:image/{key.split('/', 1)[1]};base64,{encoded}")._repr_html_()
                                        
            body += element%output.get('text/html', output.get('text/plain', str(object)))                
        return {Row: row, Column: column}[type(self)]%body

    def __iter__(self):
        for object in super().__iter__(): yield ({Row, Column} - {type(self)}).pop()(object) if isinstance(object, list) else object

class Row(Container): "A flexbox row"
class Column(Container): "A flexbox column"